In [137]:
import numpy as np
import tensorflow as tf

from matplotlib.pyplot import imshow

from tensorflow.keras.layers import *
from tensorflow.keras import Model

from PIL import Image

from captcha.image import ImageCaptcha, random_color
import random

from tqdm import tqdm

In [138]:
CAPTCHA_LEN = 4
BIN_LEN = 4
IMG_H = 60
IMG_W = 160

In [139]:
def num2vec(raw_num: str):
    """
    Convert a number captcha to vector with length CAPTCHA_LEN * BIN_LEN
    :param raw_num: number for conversion
    :return: converted vector
    """
    res = ""
    for i in raw_num:
        res += format(int(i), '04b')

    lst = []
    for i in res:
        lst.append(int(i))
    return lst

def vec2num(raw_lst: list):
    """
    Convert a vector to number
    :param raw_lst:
    :return:
    """
    p = 0
    r = ""
    while p < len(raw_lst):
        t = raw_lst[p:p+BIN_LEN]
        ts = ""
        for i in t:
            if i > 0:
                ts += "1"
            else:
                ts += "0"
        p += 4
        r += str(int(ts, 2))

    print(r)
    return r

r1 = num2vec("2834")
r2 = vec2num(r1)

2834


In [140]:
def captcha_gen() -> [Image, str]:
    image_c = ImageCaptcha()
    val = str(random.randint(1000, 9999))
    img = image_c.create_captcha_image(val, color=random_color(0, 125), background=random_color(126, 255))
    image_c.create_noise_dots(img, color=random_color(0, 255))
    image_c.create_noise_curve(img, color=random_color(0, 255))
    nparr = np.array(img)
    return nparr, val

In [141]:
# Test
img, text = captcha_gen()
num2vec(text)

[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]

In [142]:
def batch_gen(size=128):
    batch_x = np.zeros([size, IMG_H, IMG_W, 1])
    batch_y = np.zeros([size, CAPTCHA_LEN * BIN_LEN])

    for i in range(size):
        img, text = captcha_gen()
        while img.shape != (IMG_H, IMG_W, 3):
            img, text = captcha_gen()
        gray = np.mean(img, -1, keepdims=True)
        batch_x[i,:] = gray / 255
        batch_y[i,:] = num2vec(text)
    return batch_x, batch_y

x, y = batch_gen(128)

In [143]:
print(x.shape)
print(y.shape)

(128, 60, 160, 1)
(128, 16)


In [ ]:
train_batch = 10
test_batch = 1
batch_size = 128

train_X = []
train_Y = []

test_X = []
test_Y = []

for i in range(train_batch):
    tx, ty = batch_gen(batch_size)
    train_X.extend(tx)
    train_Y.extend(ty)

for i in range(test_batch):
    tx, ty = batch_gen(batch_size)
    test_X.extend(tx)
    test_Y.extend(ty)

train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)

In [ ]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

In [168]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.pool1 = MaxPooling2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(CAPTCHA_LEN * BIN_LEN)

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

model = MyModel()

In [174]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [175]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalCrossentropy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalCrossentropy(name='test_accuracy')

In [176]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    print(labels)
    print(predictions)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [177]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [178]:
EPOCHS = 5
epoch = 0
for epoch in range(EPOCHS):
    # Reset
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    train_step(train_X, train_Y)

    test_step(test_X, test_Y)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Tensor("labels:0", shape=(1280, 16), dtype=float64)
Tensor("my_model_10/dense_19/BiasAdd:0", shape=(1280, 16), dtype=float32)
Tensor("labels:0", shape=(1280, 16), dtype=float64)
Tensor("my_model_10/dense_19/BiasAdd:0", shape=(1280, 16), dtype=float32)
Epoch 1, Loss: 17.28870964050293, Accuracy: 5551.50244140625, Test Loss: 56.93853759765625, Test Accuracy: 2249.61669921875
Epoch 2, Loss: 58.1530876159668, Accuracy: 2339.409423828125, Test Loss: 56.9141845703125, Test Accuracy: 1806.8031005859375
Epoch 3, Loss: 58.22456741333008, Accuracy: 1825.0274658203125, Test Loss: 72.91291046142578, Test Accuracy: 1793.917236328125
Epoch 4, Loss: 74.10957336425781, Accuracy: 1807.146484375, Test Loss: 92.35397338867188, Test Accuracy: 1773.2412109375
Epoch 5, Loss: 93.85941314697266, Accuracy: 1785.234130859375, Test Loss: 109.14305114746094, Test Accuracy: 1756.32177734375
